In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /home/zohra/mlfs-book
HopsworksSettings initialized!


# <span style="font-width:bold; font-size: 3rem; color:#333;">Training Pipeline</span>

## 🗒️ This notebook is divided into the following sections:

1. Select features for the model and create a Feature View with the selected features
2. Create training data using the feature view
3. Train model
4. Evaluate model performance
5. Save model to model registry

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import os
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
import hopsworks
from mlfs.airquality import util
import json

import warnings
warnings.filterwarnings("ignore")

## <span style="color:#ff5f27;"> 📡 Connect to Hopsworks Feature Store </span>

In [3]:
# Check if HOPSWORKS_API_KEY env variable is set or if it is set in ~/.env
if settings.HOPSWORKS_API_KEY is not None:
    api_key = settings.HOPSWORKS_API_KEY.get_secret_value()
    os.environ['HOPSWORKS_API_KEY'] = api_key
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 

secrets = hopsworks.get_secrets_api()
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)
country=location['country']
city=location['city']
street=location['street']

2025-11-09 18:01:41,293 INFO: Initializing external client
2025-11-09 18:01:41,295 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-09 18:01:42,783 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279126


In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

--- 

## <span style="color:#ff5f27;"> 🖍 Feature View Creation and Retrieving </span>

In [5]:
# Select features for training data.
selected_features = air_quality_fg.select(['pm25', 'date']).join(weather_fg.select_features(), on=['city'])

2025-11-09 18:01:45,086 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.


In [6]:
#selected_features_df = weather_fg.read().sort_values("date")
selected_features_df=selected_features.read().sort_values("date")
selected_features_df.tail(30)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.66s) 


,pm25,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant
3687,7.0,2025-10-10 00:00:00+00:00,9.424833,0.000000,16.994516,269.810638
3688,7.0,2025-10-11 00:00:00+00:00,11.176918,0.000000,25.125961,300.047363
3689,5.0,2025-10-12 00:00:00+00:00,8.610250,0.000000,27.443359,327.559296
3690,5.0,2025-10-13 00:00:00+00:00,5.829000,0.000000,17.799076,326.045013
3691,6.0,2025-10-14 00:00:00+00:00,5.856083,0.000000,10.486448,302.239624
3692,6.0,2025-10-15 00:00:00+00:00,8.985250,0.000000,11.341428,279.035370
3693,8.0,2025-10-16 00:00:00+00:00,7.891500,0.000000,12.966248,301.601562
3694,6.0,2025-10-17 00:00:00+00:00,3.754000,0.000000,14.545515,328.634705
3695,6.0,2025-10-18 00:00:00+00:00,3.431084,0.000000,12.647482,292.290283
3696,6.0,2025-10-19 00:00:00+00:00,4.654000,0.000000,15.151039,248.033554


In [7]:
air_quality_df = air_quality_fg.read().sort_values("date")
air_quality_df.tail(30)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.17s) 


,date,pm25,country,city,street,url
1615,2025-10-10 00:00:00+00:00,7.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968
221,2025-10-11 00:00:00+00:00,7.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968
160,2025-10-12 00:00:00+00:00,5.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968
3579,2025-10-13 00:00:00+00:00,5.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968
2564,2025-10-14 00:00:00+00:00,6.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968
3448,2025-10-15 00:00:00+00:00,6.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968
1479,2025-10-16 00:00:00+00:00,8.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968
1921,2025-10-17 00:00:00+00:00,6.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968
1164,2025-10-18 00:00:00+00:00,6.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968
487,2025-10-19 00:00:00+00:00,6.0,sweden,stockholm,sollentuna-e4-haggvik,https://api.waqi.info/feed/@7968


In [8]:
weather_df = weather_fg.read().sort_values("date")
weather_df.tail(50)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.96s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
1504,2025-09-27 00:00:00+00:00,10.801917,0.000000,8.338489,14.763949,stockholm
3299,2025-09-28 00:00:00+00:00,11.864418,0.000000,14.861722,98.546646,stockholm
3358,2025-09-29 00:00:00+00:00,10.847751,0.000000,14.081477,120.567642,stockholm
3567,2025-09-30 00:00:00+00:00,10.101918,0.000000,11.734564,107.157249,stockholm
2796,2025-10-01 00:00:00+00:00,8.797750,0.400000,7.344794,115.209572,stockholm
1593,2025-10-02 00:00:00+00:00,8.624833,0.300000,7.467288,155.430206,stockholm
810,2025-10-03 00:00:00+00:00,9.074834,0.000000,14.974658,150.906647,stockholm
1492,2025-10-04 00:00:00+00:00,10.054001,15.100001,30.413076,157.591782,stockholm
3100,2025-10-05 00:00:00+00:00,10.801918,3.900000,20.707041,173.677521,stockholm
2919,2025-10-06 00:00:00+00:00,10.016501,0.000000,7.889259,314.999908,stockholm


### Feature Views

`Feature Views` are selections of features from different **Feature Groups** that make up the input and output API (or schema) for a model. A **Feature Views** can create **Training Data** and also be used in Inference to retrieve inference data.

The Feature Views allows a schema in form of a query with filters, defining a model target feature/label and additional transformation functions (declarative feature encoding).

In order to create Feature View we can use `FeatureStore.get_or_create_feature_view()` method.

You can specify the following parameters:

- `name` - name of a feature group.

- `version` - version of a feature group.

- `labels`- our target variable.

- `transformation_functions` - declarative feature encoding (not used here)

- `query` - selected features/labels for the model 

In [9]:
feature_view = fs.get_or_create_feature_view(
    name='air_quality_fv',
    description="weather features with air quality as the target",
    version=1,
    labels=['pm25'],
    query=selected_features,
)

## <span style="color:#ff5f27;">🪝 Split the training data into train/test data sets </span>

We use a time-series split here, with training data before this date `start_date_test_data` and test data after this date

In [10]:
start_date_test_data = "2025-05-01"
# Convert string to datetime object
test_start = datetime.strptime(start_date_test_data, "%Y-%m-%d")

In [11]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_start=test_start
)

Error: Reading data from Hopsworks, using Hopsworks Feature Query Service           


KeyboardInterrupt: 

In [ ]:
X_train

In [ ]:
X_features = X_train.drop(columns=['date'])
X_test_features = X_test.drop(columns=['date'])

In [ ]:
y_train

The `Feature View` is now saved in Hopsworks and you can retrieve it using `FeatureStore.get_feature_view(name='...', version=1)`.

---

## <span style="color:#ff5f27;">🧬 Modeling</span>

We will train a regression model to predict pm25 using our 4 features (wind_speed, wind_dir, temp, precipitation)

In [ ]:
xgb = XGBRegressor(
    objective="reg:squarederror",
    n_jobs=-1,
    random_state=42
)
# 2. Time series cross-validation (no shuffling)
tscv = TimeSeriesSplit(n_splits=5)

# 3. Hyperparameter search space (good starting ranges)
param_dist = {
    "n_estimators": [200, 400, 600, 800],
    "max_depth": [3, 4, 5, 6],
    "learning_rate": [0.01, 0.03, 0.05, 0.1],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "min_child_weight": [1, 3, 5],
    "gamma": [0, 0.1, 0.2],
    "reg_lambda": [1, 2, 5],
    "reg_alpha": [0, 0.1, 0.5],
}

params = {
    "n_estimators": [300, 500, 800],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 4, 5, 6],
    "subsample": [0.7, 0.9, 1.0],
    "colsample_bytree": [0.7, 0.9, 1.0],
    "min_child_weight": [1, 3, 5],
}

# 4. Search (Randomized is cheaper than full grid)
search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=40,                     # you can reduce if too slow
    scoring="neg_root_mean_squared_error",
    cv=tscv,
    verbose=1,
    n_jobs=-1, 
    random_state=42
)
# y_train as 1D
#y_train_1d = y_train.iloc[:, 0] if hasattr(y_train, "iloc") else y_train

search.fit(X_features, y_train)

print("Best params:", search.best_params_)
print("Best CV RMSE:", -search.best_score_)

# 5. Final model (already trained on full train set)
best_model = search.best_estimator_

In [ ]:
# Creating an instance of the XGBoost Regressor
xgb_regressor = XGBRegressor(**search.best_params_)

# Fitting the XGBoost Regressor to the training data
xgb_regressor.fit(X_features, y_train)


In [ ]:
# Predicting target values on the test set
y_pred = xgb_regressor.predict(X_test_features)

# Calculating Mean Squared Error (MSE) using sklearn
mse = mean_squared_error(y_test.iloc[:,0], y_pred)
print("MSE:", mse)

# Calculating R squared using sklearn
r2 = r2_score(y_test.iloc[:,0], y_pred)
print("R squared:", r2)

In [ ]:
df = y_test
df['predicted_pm25'] = y_pred

In [ ]:
df['date'] = X_test['date']
df = df.sort_values(by=['date'])
df.head(5)

In [ ]:
df.tail(5)

In [ ]:
# Creating a directory for the model artifacts if it doesn't exist
model_dir = "air_quality_model"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
images_dir = model_dir + "/images"
if not os.path.exists(images_dir):
    os.mkdir(images_dir)

In [ ]:
file_path = images_dir + "/pm25_hindcast.png"
plt = util.plot_air_quality_forecast(city, street, df, file_path, hindcast=True) 
plt.show()

In [ ]:
# Plotting feature importances using the plot_importance function from XGBoost
plot_importance(xgb_regressor)
feature_importance_path = images_dir + "/feature_importance.png"
plt.savefig(feature_importance_path)
plt.show()

---

## <span style='color:#ff5f27'>🗄 Model Registry</span>

One of the features in Hopsworks is the model registry. This is where you can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

In [ ]:
# Saving the XGBoost regressor object as a json file in the model directory
xgb_regressor.save_model(model_dir + "/model.json")

In [ ]:
res_dict = { 
        "MSE": str(mse),
        "R squared": str(r2),
    }

In [ ]:
mr = project.get_model_registry()

# Creating a Python model in the model registry named 'air_quality_xgboost_model'

aq_model = mr.python.create_model(
    name="air_quality_xgboost_model", 
    metrics= res_dict,
    feature_view=feature_view,
    description="Air Quality (PM2.5) predictor",
)

# Saving the model artifacts to the 'air_quality_model' directory in the model registry
aq_model.save(model_dir)

---
## <span style="color:#ff5f27;">⏭️ **Next:** Part 04: Batch Inference</span>

In the following notebook you will use your model for Batch Inference.
